This notebook is intended to align the NAIP to the CHM (0.6m) extent and pixel size. This may be needed if the NAIP extent is not exact or if the NAIP pixel is a different size. Otherwise, it may not be necessary and this notebook could be skipped. 

The current classification workflows do not use NAIP and CHM together. NAIP is used for bare earth percent cover and CHM is used for vegetation cover. There may be a need for using both in the future. In that case, it would be prudent to run this notebook.

In [1]:
import os
import rasterio
from rasterio.windows import from_bounds
from rasterio.enums import Resampling
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, color, transform
from skimage.feature import SIFT, match_descriptors, plot_matched_features
from skimage.measure import ransac
from skimage.transform import ProjectiveTransform, warp

def get_overlap_bounds(bounds1, bounds2):
    """Calculate the overlapping bounding box between two rasters."""
    left = max(bounds1.left, bounds2.left)
    bottom = max(bounds1.bottom, bounds2.bottom)
    right = min(bounds1.right, bounds2.right)
    top = min(bounds1.top, bounds2.top)

    if left < right and bottom < top:
        return (left, bottom, right, top)
    else:
        raise ValueError("No overlapping area between rasters.")

def get_resolution(transform):
    """Get pixel size from affine transform."""
    return abs(transform.a), abs(transform.e)

def read_and_clip_resample(path, overlap_bounds, target_res):
    with rasterio.open(path) as src:
        window = from_bounds(*overlap_bounds, transform=src.transform)
        transform = src.window_transform(window)

        # Calculate new dimensions
        width = int((overlap_bounds[2] - overlap_bounds[0]) / target_res[0])
        height = int((overlap_bounds[3] - overlap_bounds[1]) / target_res[1])

        # Read and resample
        data = src.read(
            window=window,
            out_shape=(height, width),
            resampling=Resampling.bilinear
        )

        new_transform = rasterio.transform.from_origin(
            overlap_bounds[0], overlap_bounds[3], target_res[0], target_res[1]
        )

        profile = src.profile
        profile.update({
            "height": height,
            "width": width,
            "transform": new_transform
        })

    return data, profile

In [3]:
import mpld3
mpld3.enable_notebook()  # Enable mpld3 for interactive plots in Jupyter

In [3]:
name = "Lacey"

In [4]:
os.chdir(f"../data/{name}")
os.getcwd()

'/media/grendel/7db216a7-836f-4e8d-b439-e4f999cedb232/USGS/meadow_temp/data/Lacey'

In [5]:
naip_path = rf"naip_{name}.tif"
dsm_path = rf"dsm_{name}.tif"
chm_path = rf"chm_{name}.tif"
meadow_extent_path = r"meadow_extent.geojson"

### clipping to overlapping bounds

In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.enums import Resampling as RioResampling

def ensure_epsg3857(input_path, output_path=None):
    """
    Check if a GeoTIFF is in EPSG:3857. If not, reproject it to EPSG:3857.
    Returns the path to the (possibly reprojected) file.
    If output_path is None and reprojection is needed, appends '_3857.tif' to input_path.
    """

    dst_crs = "EPSG:3857"
    with rasterio.open(input_path) as src:
        if src.crs and src.crs.to_epsg() == 3857:
            return input_path  # Already in EPSG:3857
        if output_path is None:
            output_path = input_path.replace('.tif', '_3857.tif')
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        with rasterio.open(output_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=RioResampling.bilinear
                )
    return output_path

In [ ]:
ensure_epsg3857(naip_path)
ensure_epsg3857(chm_path)

In [ ]:
import rasterio

# Output path for reprojected and resampled NAIP
naip_matched_path = naip_path.replace('.tif', '_matched_to_chm.tif')

with rasterio.open(naip_path) as naip_src, rasterio.open(chm_path) as chm_src:
    # Prepare output profile to match CHM
    profile = naip_src.profile.copy()
    profile.update({
        'crs': chm_src.crs,
        'transform': chm_src.transform,
        'width': chm_src.width,
        'height': chm_src.height
    })

    # Prepare destination array
    naip_matched = np.zeros((naip_src.count, chm_src.height, chm_src.width), dtype=naip_src.dtypes[0])

    # Reproject and resample each band
    for i in range(naip_src.count):
        reproject(
            source=rasterio.band(naip_src, i + 1),
            destination=naip_matched[i],
            src_transform=naip_src.transform,
            src_crs=naip_src.crs,
            dst_transform=chm_src.transform,
            dst_crs=chm_src.crs,
            resampling=Resampling.bilinear
        )

    # Write out the result
    with rasterio.open(naip_matched_path, 'w', **profile) as dst:
        dst.write(naip_matched)
print(f"NAIP reprojected and resampled to match CHM written to: {naip_matched_path}")

In [ ]:
# Open both rasters to get bounds and resolutions
with rasterio.open(naip_matched_path) as r1, rasterio.open(chm_path) as r2:
    bounds1 = r1.bounds
    bounds2 = r2.bounds
    res1 = get_resolution(r1.transform)
    res2 = get_resolution(r2.transform)

    # Determine finer resolution
    target_res = (min(res1[0], res2[0]), min(res1[1], res2[1]))
    target_res = round(target_res[0], 2), round(target_res[1], 2)

    # Get overlapping bounds
    overlap_bounds = get_overlap_bounds(bounds1, bounds2)

In [8]:
overlap_bounds = round(overlap_bounds[0], 3), round(overlap_bounds[1], 3), round(overlap_bounds[2], 3), round(overlap_bounds[3], 3)
overlap_bounds

(720662.997, 4371448.253, 722858.241, 4373489.453)

In [9]:
# Adjust overlap_bounds so that its width and height are divisible by target_res
left, bottom, right, top = overlap_bounds
xres, yres = target_res

# Calculate width and height in pixels
width = (right - left) / xres
height = (top - bottom) / yres

# Floor width and height to nearest integer
width_int = int(np.floor(width))
height_int = int(np.floor(height))

# Recalculate right and top to ensure divisibility
right_adj = left + width_int * xres
top_adj = bottom + height_int * yres

overlap_bounds_div = (left, bottom, right_adj, top_adj)
print("Adjusted overlap_bounds:", overlap_bounds_div)

Adjusted overlap_bounds: (720662.997, 4371448.253, 722857.797, 4373489.453)


In [10]:
# Clip and resample both rasters
raster1_data, profile1 = read_and_clip_resample(naip_matched_path, overlap_bounds, target_res)
raster2_data, profile2 = read_and_clip_resample(chm_path, overlap_bounds, target_res)

NameError: name 'naip_matched_path' is not defined

In [ ]:
# Optional: Save outputs
with rasterio.open("NAIP_clipped_resampled.tif", "w", **profile1) as dst:
    dst.write(raster1_data)

with rasterio.open("chm_clipped_resampled.tif", "w", **profile2) as dst:
    dst.write(raster2_data)


In [ ]:
import geopandas as gpd
from rasterio.mask import mask

# Read NAIP raster and meadow extent
with rasterio.open(naip_path) as naip_src:
    naip_crs = naip_src.crs

meadow_gdf = gpd.read_file(meadow_extent_path)

# Reproject meadow extent to NAIP CRS if needed
if meadow_gdf.crs != naip_crs:
    meadow_gdf = meadow_gdf.to_crs(naip_crs)

meadow_geom = meadow_gdf.geometry.values

# Clip NAIP image to meadow extent
with rasterio.open(naip_path) as src:
    naip_clipped, naip_clipped_transform = mask(src, meadow_geom, crop=True)
    naip_clipped = np.transpose(naip_clipped, (1, 2, 0))  # (rows, cols, bands)

In [ ]:
plt.imshow(naip_clipped)